Till We worked with single dataframe
However Spark will going to bring together large number of dataframes, and hence you may need to join them together.
From This section will talk about spark joins and how to use them
You required two datarame, left and right , which one to make left and which one to right that you may take decision.
Will perform join with two operations
1. Join condition/Expression join_expr = order_df.prodid === proddf_prodid
2. Join Type                 join_type = "inner"

leftdatframe.join(rightdataframe,join_expr,join_type) #inner join is default if nothing mentioned

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import *
from pyspark.sql import functions as f
from lib.logger import Log4j

spark = SparkSession.builder \
            .master("local[3]") \
            .appName("Misc Transformations") \
            .getOrCreate()

logger = Log4j(spark)

In [3]:
orders_list = [("01", "02", 350, 1),
                   ("01", "04", 580, 1),
                   ("01", "07", 320, 2),
                   ("02", "03", 450, 1),
                   ("02", "06", 220, 1),
                   ("03", "01", 195, 1),
                   ("04", "09", 270, 3),
                   ("04", "08", 410, 2),
                   ("05", "02", 350, 1)]

In [4]:
product_list = [("01", "Scroll Mouse", 250, 20),
                    ("02", "Optical Mouse", 350, 20),
                    ("03", "Wireless Mouse", 450, 50),
                    ("04", "Wireless Keyboard", 580, 50),
                    ("05", "Standard Keyboard", 360, 10),
                    ("06", "16 GB Flash Storage", 240, 100),
                    ("07", "32 GB Flash Storage", 320, 50),
                    ("08", "64 GB Flash Storage", 430, 25)]

lets create dataframe from list

In [5]:
order_df = spark.createDataFrame(orders_list).toDF("order_id", "prod_id", "unit_price", "qty")
order_df.show()

+--------+-------+----------+---+
|order_id|prod_id|unit_price|qty|
+--------+-------+----------+---+
|      01|     02|       350|  1|
|      01|     04|       580|  1|
|      01|     07|       320|  2|
|      02|     03|       450|  1|
|      02|     06|       220|  1|
|      03|     01|       195|  1|
|      04|     09|       270|  3|
|      04|     08|       410|  2|
|      05|     02|       350|  1|
+--------+-------+----------+---+



In [6]:
product_df = spark.createDataFrame(product_list).toDF("prod_id", "prod_name", "list_price", "qty")
product_df.show()

+-------+-------------------+----------+---+
|prod_id|          prod_name|list_price|qty|
+-------+-------------------+----------+---+
|     01|       Scroll Mouse|       250| 20|
|     02|      Optical Mouse|       350| 20|
|     03|     Wireless Mouse|       450| 50|
|     04|  Wireless Keyboard|       580| 50|
|     05|  Standard Keyboard|       360| 10|
|     06|16 GB Flash Storage|       240|100|
|     07|32 GB Flash Storage|       320| 50|
|     08|64 GB Flash Storage|       430| 25|
+-------+-------------------+----------+---+



In [7]:
join_expr = order_df.prod_id == product_df.prod_id

order_df.join(product_df, join_expr, "inner") \
                .show()

+--------+-------+----------+---+-------+-------------------+----------+---+
|order_id|prod_id|unit_price|qty|prod_id|          prod_name|list_price|qty|
+--------+-------+----------+---+-------+-------------------+----------+---+
|      01|     07|       320|  2|     07|32 GB Flash Storage|       320| 50|
|      03|     01|       195|  1|     01|       Scroll Mouse|       250| 20|
|      04|     08|       410|  2|     08|64 GB Flash Storage|       430| 25|
|      02|     03|       450|  1|     03|     Wireless Mouse|       450| 50|
|      01|     02|       350|  1|     02|      Optical Mouse|       350| 20|
|      05|     02|       350|  1|     02|      Optical Mouse|       350| 20|
|      02|     06|       220|  1|     06|16 GB Flash Storage|       240|100|
|      01|     04|       580|  1|     04|  Wireless Keyboard|       580| 50|
+--------+-------+----------+---+-------+-------------------+----------+---+



We get expected result.
However we not want all columns. let select specifc columns

order_df.join(product_df, join_expr, "inner") \
.select("order_id", "product_name", "unit_price", "qty") \
.show()

but both table have column with same name, it make confusion to spark and error generated
but this ambuiguity diddnt happened when we select all

order_df.join(product_df, join_expr, "inner") \
.select(*) \
.show()
It worked. why?
every column have unique id in catlog, spark internals using these id to refer columns hence select * worked. but when you use column name then create problem

1. Remove ambigues names before joining

In [9]:
product_renamed_df = product_df.withColumnRenamed("qty", "reorder_qty")

order_df.join(product_renamed_df, join_expr, "inner") \
.select("order_id", "prod_name", "unit_price", "qty") \
.show()

+--------+-------------------+----------+---+
|order_id|          prod_name|unit_price|qty|
+--------+-------------------+----------+---+
|      01|32 GB Flash Storage|       320|  2|
|      03|       Scroll Mouse|       195|  1|
|      04|64 GB Flash Storage|       410|  2|
|      02|     Wireless Mouse|       450|  1|
|      01|      Optical Mouse|       350|  1|
|      05|      Optical Mouse|       350|  1|
|      02|16 GB Flash Storage|       220|  1|
|      01|  Wireless Keyboard|       580|  1|
+--------+-------------------+----------+---+



2. drop ambigues column after join (we still have prod_id as ambigues column)

order_df.join(product_renamed_df, join_expr, "inner") \
.select("order_id","prod_name", "product_name", "unit_price", "qty") \
.show()
gets error

In [10]:
order_df.join(product_renamed_df, join_expr, "inner") \
        .drop(product_renamed_df.prod_id) \
        .select("order_id", "prod_id", "prod_name", "unit_price", "list_price", "qty") \
        .show()

+--------+-------+-------------------+----------+----------+---+
|order_id|prod_id|          prod_name|unit_price|list_price|qty|
+--------+-------+-------------------+----------+----------+---+
|      01|     07|32 GB Flash Storage|       320|       320|  2|
|      03|     01|       Scroll Mouse|       195|       250|  1|
|      04|     08|64 GB Flash Storage|       410|       430|  2|
|      02|     03|     Wireless Mouse|       450|       450|  1|
|      01|     02|      Optical Mouse|       350|       350|  1|
|      05|     02|      Optical Mouse|       350|       350|  1|
|      02|     06|16 GB Flash Storage|       220|       240|  1|
|      01|     04|  Wireless Keyboard|       580|       580|  1|
+--------+-------+-------------------+----------+----------+---+



In [11]:
spark.stop()